In [7]:
import sys
sys.path.append("../")
from initial_dataSet import DataSet
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import torch
from tqdm import tqdm

%matplotlib inline

In [8]:
basedir = '../'
baseline_list=('DINA','NCD')
baseline=baseline_list[0]

dataSet_list = ('ASSIST_0910', 'ASSIST_2017', 'JUNYI', 'MathEC', 'KDDCUP')
save_list = ('a0910/', 'a2017/', 'junyi/', 'math_ec/', 'kddcup/')

dataSet_idx=4
data_set_name = dataSet_list[dataSet_idx]
dataSet = DataSet(basedir, data_set_name)

DataSet: KDDCUP


In [9]:
def get_item_concept_df(item:pd.DataFrame) -> pd.DataFrame:
    item = item[~item.index.duplicated()]  # 去除重复项
    item_list, concept_list = [], []
    for idx in item.index:
        now_concept_list = eval(item.loc[idx, 'knowledge_code'])
        item_list.extend([idx] * len(now_concept_list))
        concept_list.extend(now_concept_list)
    return pd.DataFrame({'item': item_list, 'concept': concept_list}).astype('int')

In [10]:
read_dir='./'+baseline+'/output/'+save_list[dataSet_idx]

cogn_state=np.loadtxt(read_dir+'cognitive_state.csv',delimiter=',')
record=dataSet.record.reset_index()
item_conc=get_item_concept_df(dataSet.item)

In [11]:
doa_list=[]
for k in tqdm(range(1,cogn_state.shape[1]+1)):
    cong_doa_count=0
    item_cong_doa_count=0
    item_k=item_conc[item_conc['concept']==k-1]['item'].unique()
    record_k=record.set_index('item_id').loc[item_k,:]
    if len(item_k)>0 and len(record_k)>0:
        for j in item_k:
            record_j=record_k.loc[j,:]
            stu_j=np.array(record_j['user_id']).reshape(-1).astype('int')
            cong_j=cogn_state[stu_j-1,k-1]
            sort_stu_idx=cong_j.argsort() # 从小到大
            sort_score_j=np.array(record_j['score']).reshape(-1)[sort_stu_idx]
            sort_cong_j=cong_j[sort_stu_idx]
            for i in range(len(sort_stu_idx)):
                fliter=((sort_cong_j[i:]-sort_cong_j[i])>=0)
                cong_doa_count+=fliter.sum()
                score_i=sort_score_j[i:][fliter]
                if len(score_i)>0:
                    item_cong_doa_count+=((score_i-sort_score_j[i])>=0).sum()
        if cong_doa_count>0:
            doa_list.append(item_cong_doa_count/cong_doa_count)

100%|██████████| 16/16 [00:21<00:00,  1.35s/it]


In [12]:
print(baseline)
print('{} 的DOA = {}'.format(data_set_name,np.mean(doa_list)))

DINA
KDDCUP 的DOA = 0.9052007940936431
